In [49]:
# goals:
# for each neural dataset, get the data into a reasonable format

## import relevant libraries

In [104]:
import numpy as np
import pandas as pd
import nibabel as nib
import os
from os import listdir
from os.path import isfile, join, exists
import matplotlib.pyplot as plt
import nilearn.plotting as plotting
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.spatial.distance import pdist, squareform,cosine,euclidean,mahalanobis
from statsmodels.stats.multitest import multipletests
import scipy.io as sio
import scipy.stats as stats
from IPython.core.debugger import set_trace
from tqdm import tqdm
import time


## hyperparameters

In [105]:
home_dir = '/home/jacobpri/git/RotspaceIT/'
dataset = 'Cohen2017'

## dataset-specific opts

In [106]:
opts = dict()
if dataset == 'Cohen2017':
    opts['datatypes_avail'] = ['rdv']
    opts['levels_avail'] = ['categ']
    opts['data_fn'] = 'RoiRsa_reordered.mat'
    opts['struct_name'] = 'R'
    opts['roi_fieldname'] = 'roiList'
    opts['data_fieldname'] = 'roiRsaSubCube'
    opts['categ_strs'] = ['faces','cats','bodies','cars','hammers','phones','chairs','buildings']
    opts['plot_order'] = np.array([0,2,1,4,6,7,5,3])
elif dataset == 'Baker2018':
    opts['data_avail'] = ['categ','item']

## load fMRI data

In [109]:
data_dir = join(home_dir,'data','d01_rawdata',dataset)
save_dir = join(home_dir,'data','d03_processing',dataset)

assert(exists(data_dir))
if not exists(save_dir):
    os.makedirs(save_dir)
    
fmri = dict()

assert(exists(data_dir))

if dataset == 'Cohen2017':
    
    # dataset 
    X = sio.loadmat(join(data_dir,opts['data_fn']))[opts['struct_name']]

    # 16 ROIs
    roi_list = np.concatenate(X[opts['roi_fieldname']][0][0][0])
    print(roi_list)
    
    # ROIs x subjects x categ pairs
    roi_rsa_subcube = X[opts['data_fieldname']][0][0]
    print(roi_rsa_subcube.shape)
    
    # save this
    
    # average over subjects
    roi_rsa_submean = np.nanmean(roi_rsa_subcube,axis=1)
    print(roi_rsa_submean.shape)
    
    # dictionary of category RDVs by roi. fmri[roi][data][level] (e.g. fmri['FFA']['rdv']['item'])
    for r in range(len(roi_list)):
        
        roi = roi_list[r]
        fmri[roi] = dict()
        
        for datatype in opts['datatypes_avail']:
            
            fmri[roi][datatype] = dict()
            
            for level in opts['levels_avail']:
                
                fmri[roi][datatype][level] = roi_rsa_submean[r]
    

    # save relevant files
    fn = join(save_dir,'fmri_roi_rsa_subcube.npy')
    np.save(fn, roi_rsa_subcube)
    
    fn = join(save_dir,'fmri_roi_rsa_submean.npy')
    np.save(fn, fmri)
    
    fn = join(save_dir, 'roi_list.npy')
    np.save(fn, roi_list)
    
    
elif dataset == 'Baker2017':
    
    pass

['FFA' 'OFA' 'FBA' 'EBA' 'LO' 'PPA' 'TOS' 'MT' 'VentralOTC' 'LateralOTC'
 'OPC' 'EarlyV2' 'VentralOTCRemoved' 'LateralOTCRemoved' 'OPCRemoved'
 'EarlyV2Removed']
(16, 6, 28)
(16, 28)


In [108]:
print(np.corrcoef(fmri['FFA']['rdv']['categ'],fmri['PPA']['rdv']['categ']))

[[1.         0.81185753]
 [0.81185753 1.        ]]


In [ ]:
def rsm2rdm(matrix):
    return ((matrix - 1) * -1)

fmri_dir = '/home/jacobpri/RotspaceIT/CohenfMRIData/'

Cohen = sio.loadmat(join(fmri_dir,'RoiRsa_reordered.mat'))['R']
#print(Cohen)
roi_list = Cohen['roiList'][0]

FFA_idx = 0
PPA_idx = 5
#print(roi_list)
#print(Cohen)
print(Cohen['roiRsaSubCube'][0,0].shape)
R = Cohen['roiRsaSubCube'][0,0]
print(R.shape)
R_ = np.nanmean(R,axis=1)
print(R_.shape)
subj_FFA_rdv = R_[FFA_idx,:]
subj_PPA_rdv = R_[PPA_idx,:]
print(subj_FFA_rdv.shape)
#Cohen['roiCondRSA'].shape
#plt.figure()
#plt.imshow(subj_FFA_rdv,aspect='auto',clim=(0,1.5))
#plt.colorbar()

avg_FFA_rdv = subj_FFA_rdv#np.nanmean(subj_FFA_rdv,axis=0)
avg_PPA_rdv = subj_PPA_rdv#np.nanmean(subj_PPA_rdv,axis=0)
print(avg_FFA_rdv.shape)

print(np.corrcoef(avg_FFA_rdv,avg_PPA_rdv)[0,1])

plt.figure()
plt.subplot(121)
plt.imshow(squareform(rsm2rdm(avg_FFA_rdv)))
plt.subplot(122)
plt.imshow(squareform(rsm2rdm(avg_PPA_rdv)))